<a href="https://colab.research.google.com/github/NeurologiaLogic/Artificial-intelligence/blob/main/Profanity_Filter_With_LinearSVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
import joblib
from google.colab import files
import regex as re
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV


lemmatizer = WordNetLemmatizer()
files.upload()
# Read in data
data = pd.read_csv('./labeled_data.csv')
texts = data['tweet'].astype(str)
y = data['class']

#data cleaning
for index,curr_x in enumerate(texts):
    curr_x = re.sub(r'[\d+]', '', curr_x)
    curr_x = re.sub(r'http\w*','',curr_x) 
    curr_x = re.sub(r'@\w*','',curr_x) 
    curr_x = curr_x.translate(str.maketrans("","", string.punctuation))
    curr_x = curr_x.strip()
    curr_x = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", curr_x)
    temp = []
    for word in curr_x.split(" "):
        temp.append(lemmatizer.lemmatize(word))
    curr_x = " ".join(temp)
    texts[index] = curr_x



vectorizer = CountVectorizer(stop_words='english', min_df=0.0001)
X = vectorizer.fit_transform(texts)

# Train the model Linear SVC
model = LinearSVC(class_weight="balanced", dual=False, tol=1e-2, max_iter=1e5,verbose=1)
cclf = CalibratedClassifierCV(base_estimator=model)
cclf.fit(X, y)

# Train the model SDG -> train terlalu lama
# params = {
#     "loss" : ["hinge", "log", "squared_hinge", "modified_huber", "perceptron"],
#     "alpha" : [0.001, 0.01, 0.1],
#     "penalty" : ["l2", "l1", "elasticnet", "none"],
# }

# clf = SGDClassifier(max_iter=1e6)
# grid = GridSearchCV(clf, param_grid=params, cv=10)
# grid.fit(X, trainY)


# Save the model
joblib.dump(vectorizer, 'vectorizer.joblib')
joblib.dump(cclf, 'modelSVC.joblib')
# joblib.dump(grid, 'modelSDG.joblib')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

['modelSVC.joblib']

In [ ]:
import joblib
import numpy as np

vectorizer = None
model = None

vectorizer=joblib.load('./vectorizer.joblib')

model=joblib.load('./modelSVC.joblib')


def _get_profane_prob(prob):
  return prob[1]

def predict(texts):
  return model.predict(vectorizer.transform(texts))

def predict_prob(texts):
  return np.apply_along_axis(_get_profane_prob, 1, model.predict_proba(vectorizer.transform(texts)))


In [ ]:
#evaluating the model
# from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,classification_report
# from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(texts, y, test_size = 0.2)
# _,X_test,_,y_test = train_test_split(texts,y,test_size=0.25,random_state=42)
predicted = [predict([x_test]) for x_test in testX]
# Evaluating the model
acc_naive = accuracy_score(testY,predicted)
print(f'model has {acc_naive.round(2)*100}% accuracy')


model has 96.0% accuracy


In [ ]:
import spacy
from spacy import displacy
from spacy.tokens import Span
#0 tidak bisa di definisi kasar atau biasa
#1 buat kasar 
#2 buat biasa 

nlp = spacy.load("en_core_web_sm")
sentences = "You are so dumb that looking at you made me want to puke like shit"

doc = nlp(sentences)
words = sentences.split(" ")
options = {}
doc.spans["sc"] = []
for i,v in enumerate(words):
  result = "1" if predict([v])==1 else "2"
  if(result=="1"):
    doc.spans["sc"].append(Span(doc, i, i+1, "BW"))
displacy.render(doc, style="span",options = options,jupyter=True)



